# Notebook for ANNDL - Homework 1

Team: All Is Well

Team members: Fatma Hamila, Kodai Takigawa, Zheng Maria Yu

# Introduction
This notebook is used to ensemble K-Fold cross validation models.

In [1]:
# Import libraries
import tensorflow as tf
import numpy as np
import os
import random

tfk = tf.keras
tfkl = tf.keras.layers

In [2]:
from PIL import Image
import os
from albumentations import *
import albumentations as alb

class PlantDataset():
    """The class PlantDataset loads the data and executes the pre-processing operations on it"""

    def __init__(self, path,n_fold,i,one_hot=False):
        images = self.load_images(path)
        self.images1, self.labels1, self.images2,self.labels2 = self.split_images(images, n_fold,i)
        if one_hot:
            self.labels1=self.onehot(self.labels1)
            self.labels2=self.onehot(self.labels2)

    @staticmethod
    def split_images(images, n_fold,i):
        train_data = []
        train_labels = []
        test_data = []
        test_labels = []
        for k in images.keys():
            n = len(images[k])//n_fold
            train_data += images[k][:i*n]+images[k][(i+1)*n:]
            test_data += images[k][i*n:(i+1)*n]
            train_labels += [k] * (len(images[k]) - n)
            test_labels += [k] * (n)
        return np.asarray(train_data), np.asarray(train_labels), np.asarray(test_data), np.asarray(test_labels)

    @staticmethod
    def onehot(x,n_class=8):
        y=np.zeros((len(x),n_class))
        for i in range(n_class):
            y[np.where(x==i),i]=1
        return y

    def get_images(self):
        return self.images1, self.labels1,self.images2, self.labels2

    @staticmethod
    def load_images(image_path):
        """This method loads the images from the given path"""
        images = {}
        for i, file in enumerate(os.listdir(image_path)):
            for img in os.listdir(os.path.join(image_path, file)):
                path = os.path.join(image_path, file, img)
                image = Image.open(path)
                if i in images:
                    images[i].append(np.asarray(image))
                else:
                    images[i]=[np.asarray(image)]

        return images

    def __len__(self):
        return len(self.images1)

ds=PlantDataset("data/train/",n_fold=1,i=0,one_hot=True)
train_data,train_labels,test_data,test_labels=ds.get_images()
test_data.shape

(2607, 96, 96, 3)

In [3]:
# Fix random seed for reproducibility
seed = 23

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

In [10]:
# Define categorical focal loss
from keras import backend as K
def categorical_focal_loss(y_true, y_pred):
        """
        :param y_true: A tensor of the same shape as `y_pred`
        :param y_pred: A tensor resulting from a softmax
        :return: Output tensor.
        """
        alpha = np.array([0.25,0.1,0.1,0.1,0.1,0.1,0.1,0.15], dtype=np.float32)
        gamma=2
        # Clip the prediction value to prevent NaN's and Inf's
        epsilon = K.epsilon()
        y_pred = K.clip(y_pred, epsilon, 1. - epsilon)

        # Calculate Cross Entropy
        cross_entropy = -y_true * K.log(y_pred)

        # Calculate Focal Loss
        loss = alpha * K.pow(1 - y_pred, gamma) * cross_entropy

        # Compute mean loss in mini_batch
        return K.mean(K.sum(loss, axis=-1))

# Model evaluation

In [14]:
ds=PlantDataset("training_data_final/",n_fold=1,i=0)
train_data,train_labels,test_data,test_labels=ds.get_images()
print(test_data.shape,test_labels.shape)
data=np.asarray(test_data)
data.shape

(3264, 96, 96, 3) (3264,)


(3264, 96, 96, 3)

In [18]:
def predict(model, X):
    X = np.reshape(X, (-1, 96, 96, 3))
    assert X.ndim == 4
    from tensorflow.keras.applications.efficientnet_v2 import preprocess_input
    X = preprocess_input(X)
    X=tf.image.resize(X,(224,224),method='gaussian')
    prediction = model.predict(X)
    output = tf.argmax(prediction, axis=-1)
    return output,prediction



def accuracy(ds, model,step=100,data=None):
    """Compute the accuracy rate on the given dataset with the input model"""
    from tqdm import tqdm

    num_correct = 0
    if data is not None:
        data,target=data
    else:
        data,target=ds.get_images()
    loop=tqdm(range(0,len(data)//step+1))
    tp=[0]*8
    tn=[0]*8
    fp=[0]*8
    fn=[0]*8
    for i in loop:
        x=data[i*step:min(len(data),(i+1)*step)]
        y=target[i*step:min(len(data),(i+1)*step)]
        output,_ = predict(model,x)
        c=np.array(y==output)
        num_correct += sum(c)
        for a,b in zip(y,output):
            if a==b:
                tp[a]+=1
                for i in range(8):
                    if i==a:
                        continue
                    tn[i]+=1
            else:
                fp[b]+=1
                fn[a]+=1
                for i in range(8):
                    if i==a or i==b:
                        continue
                    tn[i]+=1


    print(num_correct/len(data))
    f1=[]
    for i in range(8):
        precision=tp[i]/(tp[i]+fp[i]+1e-3)
        recall=tp[i]/(tp[i]+fn[i]+1e-3)
        f1.append(2 * precision * recall / (precision + recall + 1e-3))
        print("class:",i,"f1 score:",2 * precision * recall / (precision + recall + 1e-3),"precision:",precision,"recall:",recall)
    print("overall:",sum(f1)/8)


In [13]:
# Load single models
model = tfk.models.load_model("Final/KF1.ckpt")
#accuracy(ds,model,data=(data,test_labels))

In [19]:
# Evaluate the performance of single models
accuracy(ds,model,data=(data,test_labels))

100%|██████████| 33/33 [01:15<00:00,  2.27s/it]

0.9954044117647058
class: 0 f1 score: 0.9888567486842236 precision: 0.9789422160935994 recall: 0.9999946236848188
class: 1 f1 score: 0.9947769472249179 precision: 0.9999981024703939 recall: 0.9905996417299968
class: 2 f1 score: 0.995622353261903 precision: 0.9941953690611818 recall: 0.9980563144537584
class: 3 f1 score: 0.9955844072479714 precision: 0.9960841563910834 recall: 0.9960841563910834
class: 4 f1 score: 0.9938379996080113 precision: 0.9962173984548234 recall: 0.9924651742652086
class: 5 f1 score: 0.9972384121821163 precision: 0.9999954751335967 recall: 0.9954910113017509
class: 6 f1 score: 0.9976361942224039 precision: 0.998135943880924 recall: 0.998135943880924
class: 7 f1 score: 0.9908379612547786 precision: 0.9870647109279701 recall: 0.9956478450093695
overall: 0.9942988779607908


# Model Ensembling
Here we define the model ensembling procedure.

In [ ]:
# Ensemble the models together
class Model(tf.keras.Model):
    def __init__(self):
        super(Model, self).__init__()
        self.model0=tfk.models.load_model("Final/KF_0.ckpt")
        self.model1=tfk.models.load_model("Final/KF_1.ckpt")
        self.model2=tfk.models.load_model("Final/KF_2.ckpt")
        self.model3=tfk.models.load_model("Final/KF_3.ckpt")
        self.model4=tfk.models.load_model("Final/KF_4.ckpt")


    def call(self,x):
        y=self.model0(x)
        y+=self.model1(x)
        y+=self.model2(x)
        y+=self.model3(x)
        y+=self.model4(x)

        return y/5

    def predict(self,x):
        y=self.model0.predict(x)
        y+=self.model1.predict(x)
        y+=self.model2.predict(x)
        y+=self.model3.predict(x)
        y+=self.model4.predict(x)

        return y/5

model=Model()
#accuracy(ds,model,data=(data,test_labels))

In [12]:
# Compile and save the final model
loss=tfk.losses.CategoricalCrossentropy()
optimizer = tfk.optimizers.SGD(learning_rate=1e-1)
metrics = ['accuracy']
input_layer = tfkl.Input(shape=(224,224,3), name='Input')
x=model(input_layer)
model = tfk.Model(inputs=input_layer, outputs=x)
model.compile(
        optimizer=optimizer,
        loss=loss,
        metrics=metrics
    )
model.save('Final/KF1.ckpt')

INFO:tensorflow:Assets written to: Final/KF1.ckpt\assets


INFO:tensorflow:Assets written to: Final/KF1.ckpt\assets


In [44]:
# Choose the best models to ensemble together
for n in [16,18,27,28,34,90]:
    print(n)
    model = tfk.models.load_model("experiments/FT_KF_EFF_B0_4Cos_0_Nov23_10-56-11"+"/base_ckpts/cp_"+str(n)+".ckpt",
                                  custom_objects={"categorical_focal_loss": categorical_focal_loss})
    accuracy(ds,model,data=(data,test_labels))

16


100%|██████████| 7/7 [00:02<00:00,  3.00it/s]


0.8569254185692542
class: 0 f1 score: 0.6217208758828198 precision: 0.538451183631084 recall: 0.7368227151917055
class: 1 f1 score: 0.8664882867488837 precision: 0.9166571181550192 recall: 0.8224222203530808
class: 2 f1 score: 0.8947300295443079 precision: 0.8784964626498818 recall: 0.9126124989077775
class: 3 f1 score: 0.8818446219209709 precision: 0.8910802863337987 recall: 0.87377792448617
class: 4 f1 score: 0.871052091961182 precision: 0.8558481455122026 recall: 0.8878421696993486
class: 5 f1 score: 0.8935986170395227 precision: 0.9499762505937352 recall: 0.8444256794293461
class: 6 f1 score: 0.9478482653943285 precision: 0.9618956009942762 recall: 0.9351765261432764
class: 7 f1 score: 0.636849081924881 precision: 0.6444301237750273 recall: 0.6304210778026565
overall: 0.826766483802112
18


100%|██████████| 7/7 [00:02<00:00,  2.85it/s]


0.863013698630137
class: 0 f1 score: 0.5864592845378112 precision: 0.49999074091220536 recall: 0.7105076182205732
class: 1 f1 score: 0.8757852190421765 precision: 0.9770002643647774 recall: 0.7943850992046803
class: 2 f1 score: 0.9225683642888186 precision: 0.9142770068856486 recall: 0.9320297861185813
class: 3 f1 score: 0.872040799647595 precision: 0.8811793942634231 recall: 0.8640692808807681
class: 4 f1 score: 0.8762048676579934 precision: 0.8571352041499629 recall: 0.8971878767488154
class: 5 f1 score: 0.9190199571059905 precision: 0.9523582771838767 recall: 0.888869136241417
class: 6 f1 score: 0.9669331301773257 precision: 0.97195353314455 recall: 0.9629540467217895
class: 7 f1 score: 0.6310665445838176 precision: 0.6122324034203384 recall: 0.6521597356579205
overall: 0.8312597708801911
27


...

0.867579908675799
class: 0 f1 score: 0.6201837692386253 precision: 0.5510091630783045 recall: 0.7105076182205732
class: 1 f1 score: 0.8794939311965403 precision: 0.9462263846625305 recall: 0.8224222203530808
class: 2 f1 score: 0.9233011773883858 precision: 0.9065335837982822 recall: 0.9417384297239833
class: 3 f1 score: 0.8782801018556415 precision: 0.9157798338964852 recall: 0.8446519936699644
class: 4 f1 score: 0.8773208959075021 precision: 0.8508697292129016 recall: 0.9065335837982822
class: 5 f1 score: 0.8935986170395227 precision: 0.9499762505937352 recall: 0.8444256794293461
class: 6 f1 score: 0.9478482653943285 precision: 0.9618956009942762 recall: 0.9351765261432764
class: 7 f1 score: 0.6994895524952208 precision: 0.6481361456269329 recall: 0.7608530249342406
overall: 0.8399395388144708
34


100%|██████████| 7/7 [00:02<00:00,  2.96it/s]


0.8751902587519026
class: 0 f1 score: 0.5778033001205505 precision: 0.5333214817448502 recall: 0.6315623273071762
class: 1 f1 score: 0.8994937249540469 precision: 0.9677315297684971 recall: 0.8411136344520145
class: 2 f1 score: 0.9127357025933729 precision: 0.8620615339522935 recall: 0.9708643605401889
class: 3 f1 score: 0.9004816515634985 precision: 0.9191826345188432 recall: 0.8834865680915719
class: 4 f1 score: 0.8937228659346326 precision: 0.9207829625449253 recall: 0.8691507556004149
class: 5 f1 score: 0.8883694173395458 precision: 0.888869136241417 recall: 0.888869136241417
class: 6 f1 score: 0.9523216398504022 precision: 0.9711445082258824 recall: 0.9351765261432764
class: 7 f1 score: 0.7194891427226576 precision: 0.6666543212162738 recall: 0.7825916827895046
overall: 0.8430521806348384
90


100%|██████████| 7/7 [00:02<00:00,  2.85it/s]

0.8858447488584474
class: 0 f1 score: 0.6105960856477562 precision: 0.6470397929472663 recall: 0.5789321333649114
class: 1 f1 score: 0.9049659923645027 precision: 0.9680748077148115 recall: 0.8504593415014813
class: 2 f1 score: 0.9340716702975307 precision: 0.9008927847496869 recall: 0.9708643605401889
class: 3 f1 score: 0.8775405106886773 precision: 0.8823442907422476 recall: 0.87377792448617
class: 4 f1 score: 0.8980428922746311 precision: 0.929990700092999 recall: 0.8691507556004149
class: 5 f1 score: 0.9445344997865772 precision: 0.9347622877763527 recall: 0.9555343214595232
class: 6 f1 score: 0.9406684743737991 precision: 0.9203458376474544 recall: 0.9629540467217895
class: 7 f1 score: 0.7567728741079278 precision: 0.6841985228329328 recall: 0.8478076563552966
overall: 0.8583991249426753


# Model saving
The model is saved and it can be reloaded if necessary.

In [ ]:
# Save the model
model.save(mode)

INFO:tensorflow:Assets written to: Baseline/assets


INFO:tensorflow:Assets written to: Baseline/assets
